In [6]:
!pip install torch==1.8.1 torchvision==0.9.1 torchaudio==0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |█████████████▌                  | 834.1 MB 1.5 MB/s eta 0:12:23tcmalloc: large alloc 1147494400 bytes == 0x55b7d5a4a000 @  0x7f14be3c3615 0x55b79d31e4cc 0x55b79d3fe47a 0x55b79d3212ed 0x55b79d412e1d 0x55b79d394e99 0x55b79d38f9ee 0x55b79d322bda 0x55b79d394d00 0x55b79d38f9ee 0x55b79d322bda 0x55b79d391737 0x55b79d413c66 0x55b79d390daf 0x55b79d413c66 0x55b79d390daf 0x55b79d413c66 0x55b79d390daf 0x55b79d323039 0x55b79d366409 0x55b79d321c52 0x55b79d394c25 0x55b79d38f9ee 0x55b79d322bda 0x55b79d391737 0x55b79d38f9ee 0x55b79d322bda 0x55b79d390915 0x55b79d322afa 0x55b79d390c0d 0x55b79d38f9ee
     |█████████████████               | 1055.7 MB 1.4 MB/s eta 0:10:56tcmalloc: large alloc 1434370048 bytes == 0x55b81a0a0000 @  0x7f14be3c3615 0x55b79d31e4cc 0x55b79d3fe47a 0x55b79d3212ed 0x55b79d412e1d 0x55b79d394e99 0x55b79d38f9ee 0x55b79d322bda 0x55b79d394d00 0x55b79d38f9ee 0x55b79d322bda 0x55b79d391737 0x55b79d413c66 0x55b79

In [7]:
!pip install transformers ipywidgets

     |████████████████████████████████| 2.8 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 
     |████████████████████████████████| 636 kB 56.8 MB/s 
     |████████████████████████████████| 895 kB 58.1 MB/s 
     |████████████████████████████████| 52 kB 837 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [23]:
import pandas as pd
import zipfile
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chrisdallago","key":"47505734214414419614bf9292bbbfbf"}'}

In [11]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
!kaggle competitions download -c umich-siads-695-predicting-text-difficulty

  0% 0.00/278k [00:00<?, ?B/s]
100% 278k/278k [00:00<00:00, 38.4MB/s]
  0% 0.00/410k [00:00<?, ?B/s]
100% 410k/410k [00:00<00:00, 55.9MB/s]
  0% 0.00/2.83k [00:00<?, ?B/s]
100% 2.83k/2.83k [00:00<00:00, 2.86MB/s]
 44% 9.00M/20.3M [00:00<00:00, 90.5MB/s]
100% 20.3M/20.3M [00:00<00:00, 130MB/s] 
  0% 0.00/834k [00:00<?, ?B/s]
100% 834k/834k [00:00<00:00, 117MB/s]
  0% 0.00/4.73M [00:00<?, ?B/s]
100% 4.73M/4.73M [00:00<00:00, 150MB/s]
  0% 0.00/18.3k [00:00<?, ?B/s]
100% 18.3k/18.3k [00:00<00:00, 19.3MB/s]


In [13]:
!ls

additional_resource_file_readme.txt		  sample_data
AoA_51715_words.csv.zip				  sampleSubmission.csv.zip
Concreteness_ratings_Brysbaert_et_al_BRM.txt.zip  WikiLarge_Test.csv.zip
dale_chall.txt					  WikiLarge_Train.csv.zip
kaggle.json


In [16]:
zf = zipfile.ZipFile('/content/WikiLarge_Train.csv.zip') 
train_df = pd.read_csv(zf.open('WikiLarge_Train.csv'))

In [17]:
tenK_train = train_df.sample(10000)
en_X_train = tenK_train.original_text

In [18]:
# Download the tokenizers and models from through the HuggingFace Transformer
# Italian
it_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-it")
it_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-it")
# Russian
ru_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
ru_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
# Simplified Chinese
zh_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-zh")
zh_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-zh")

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/789k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/814k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/806k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/805k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

In [19]:
# Create a translation pipeline for each language using a tokenizer and model for each
en_to_it_translation_pipeline = pipeline('translation_en_to_it', tokenizer=it_tokenizer, model=it_model)
en_to_ru_translation_pipeline = pipeline('translation_en_to_ru', tokenizer=ru_tokenizer, model=ru_model)
en_to_zh_translation_pipeline = pipeline('translation_en_to_zh', tokenizer=zh_tokenizer, model=zh_model)

In [20]:
# Initiate the translation process
# English to Italian
it_X_train = []
for sent in tqdm(en_X_train):
    result = en_to_it_translation_pipeline(sent)
    it_X_train.append(result[0]['translation_text'])

# English to Russian
ru_X_train = []
for sent in tqdm(en_X_train):
    result = en_to_ru_translation_pipeline(sent)
    ru_X_train.append(result[0]['translation_text'])

# English to Chinese
zh_X_train = []
for sent in tqdm(en_X_train):
    result = en_to_zh_translation_pipeline(sent)
    zh_X_train.append(result[0]['translation_text'])


100%|██████████| 100/100 [02:34<00:00,  1.54s/it]


In [24]:
# Create a Pandas DataFrame with all translations
translate_df = pd.DataFrame({'english':en_X_train, 'italian':it_X_train, 'russian':ru_X_train, 'chinese_s':zh_X_train})
translate_df.head()

,english,italian,russian,chinese_s
126173,Retrieved on 21 February 2008 .,URL consultato il 21 febbraio 2008 .,Вернут 21 февраля 2008 года.,2008年2月21日检索
49386,Most earthquake clusters consist of small trem...,La maggior parte dei gruppi sismici consiste d...,Большинство землетрясений состоят из небольших...,"大部分地震群由小震震组成, 几乎不会造成任何损害,"
148089,Often considered one of the greatest contribut...,Spesso considerato uno dei maggiori contributo...,Часто считался одним из величайших вкладчиков ...,"在《百科全书》(百科全书)中, “Fuzuli, Mehmed bin Süleyman” ..."
288754,Corporate,Corporate,Корпоративная деятельность,公司
292659,Europe Undergraduate degrees are not like in t...,Laureati in Europa non sono come negli Stati U...,"В Европе степень бакалавра не такая, как в Сое...","欧洲大学本科学位与美国不同, 学生在第一年有一般研究, 在大学最后几年只专门攻读“主修”专业。"


In [26]:
# Create stopword lists for each language and combine with English stopwords
en_stopwords_list = stopwords.words('english')
it_stopwords_list = stopwords.words('english') + stopwords.words('italian')
ru_stopwords_list = stopwords.words('english') + stopwords.words('russian')

with open('chinese_simplified_stopwords.txt', 'r') as wl:
    words = [word for word in wl]
chinese_stop_words = []
for w in words:
    if '\n' in w:
        w = w[:-1]
        chinese_stop_words.append(w)
        
zh_stopwords_list = stopwords.words('english') + chinese_stop_words

In [29]:
# Vectorize and get the mean cross validation score for each language
def get_score(language, stopword_list):
    vectorizer = TfidfVectorizer(min_df=25, stop_words=stopword_list, ngram_range=(1, 2)) 
    X_train_vec = vectorizer.fit_transform(translate_df[language])
    y_train = tenK_train.label
    
    clf = LogisticRegression()
    shuff_spt = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=10)
    scores = cross_val_score(clf, X_train_vec, y_train, cv=shuff_spt) 
    scores.mean()

    return scores.mean()

In [30]:
# View scores in Pandas DataFrame
en_score = get_score('english', en_stopwords_list)
it_score = get_score('italian', it_stopwords_list)
ru_score = get_score('russian', ru_stopwords_list)
zh_score = get_score('chinese_s', zh_stopwords_list)

score_df = pd.DataFrame({'english':[en_score], 'italian':[it_score], 'russian':[ru_score],'chinese_s':[zh_score]}).T.rename(columns={0:'accuracy_score'})
score_df

,accuracy_score
english,0.466
italian,0.448
russian,0.472
chinese_s,0.476
